In [74]:
#!git clone https://github.com/uestudante/Bressay.git  ## to acess the data set.

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
import requests
import simplejson
import base64
import os
from pathlib import Path

MAKE DIRECTORIES

In [85]:

# os.makedirs('/content/drive', exist_ok=True)

models = ['openai/gpt-4o-mini', 'anthropic/claude-3.7-sonnet', 'google/gemini-2.0-flash-001',
       'google/gemini-2.5-flash-preview', 'google/gemini-2.5-pro-preview', 'deepseek/deepseek-chat-v3-0324:free',
        'deepseek/deepseek-chat-v3-0324', 'google/gemini-2.5-flash-preview:thinking',
        'anthropic/claude-3.7-sonnet:thinking', 'openai/gpt-4.1']

# diretorios = []

#for model in models:
#    model = model.replace('/', '-')
#    path = os.path.join('/content/Bressay/resultados', model)
#    diretorios.append(path)
#    os.makedirs(path, exist_ok=True)


#partial_path_model = []     ## caminho para guardar resultado de cada modelo

#models = ['anthropic/claude-3.7-sonnet'] ##modelos para pesquisa

#for model in models:
#    model = model.replace('/', '-')
#    partial_path_model.append(model)

#print(partial_path_model)

In [77]:
data_set = os.listdir('/content/drive/MyDrive/projeto-correcao-provas/.bressay/data/pages')
#print(data_set)

output_file_path = '/content/drive/MyDrive/projeto-correcao-provas/.bressay/sets/benchmarking.txt'

output_directory = os.path.dirname(output_file_path)
os.makedirs(output_directory, exist_ok=True)
print(f"Verificando/criando diretório: {output_directory}")


# Salva as strings no arquivo
try:
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for item in data_set:
            f.write(f"{item}\n")
    print(f"Strings salvas com sucesso em '{output_file_path}'")

except Exception as e:
    print(f"Ocorreu um erro ao salvar o arquivo: {e}")


Verificando/criando diretório: /content/drive/MyDrive/projeto-correcao-provas/.bressay/sets
Strings salvas com sucesso em '/content/drive/MyDrive/projeto-correcao-provas/.bressay/sets/benchmarking.txt'


In [84]:
## Using openrouter to benchmarking

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')


def openrouter_request(image_path, model):
    url = "https://openrouter.ai/api/v1/chat/completions"

    headers = {
        "Authorization": f"Bearer {'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'}",
        "Content-Type": "application/json"
    }


    base64_image = encode_image_to_base64(image_path)
    data_url = f"data:image/png;base64,{base64_image}"

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "Only Transcribe into Brazilian Portuguese"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": data_url
                    }
                }
            ]
        }
    ]

    payload = {
        "model": model,
        "messages": messages
    }

    response = requests.post(url, headers=headers, json=payload)
    #print(response.json())


    text_json = response.json()
    text_content = text_json['choices'][0]['message']['content']
    #print(text_content)

    return text_content



In [79]:
def benchmarking(image_path, model):
    result = openrouter_request(image_path, model)
    #result = ''
    model = model.replace('/', '-')
    image_name = os.path.basename(image_path)[:-4]
    path = os.path.join('/content/drive/MyDrive/projeto-correcao-provas/benchmarking', model)


    with open(f'{path}_{image_name}.txt', 'w', encoding='utf-8') as file:
        file.write(result)


In [80]:
##
num_lines_to_read = 1
arquivos_test = []

with open(r'/content/drive/MyDrive/projeto-correcao-provas/.bressay/sets/benchmarking.txt', 'r') as file:
  for i in range(num_lines_to_read):
      arquivo = file.readline()
      if not arquivo:
        break
      arquivos_test.append(arquivo.strip())



In [81]:
data_directory = r'/content/drive/MyDrive/projeto-correcao-provas/.bressay/data/pages'

arquivos_data = []

for dir_name in arquivos_test:
    full_path = os.path.join(data_directory, dir_name, dir_name) + '.png'
    arquivos_data.append(full_path)



In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import os

# Diretório onde seus arquivos TXT (exceto o gabarito) estão localizados
diretorio_textos = '/content/drive/MyDrive/projeto-correcao-provas/benchmarking'

# Caminho completo para o arquivo do gabarito
# Substitua 'nome_do_arquivo_gabarito.txt' pelo nome real do seu arquivo gabarito
caminho_gabarito = '/content/drive/MyDrive/projeto-correcao-provas/benchmarking/gabarito.txt'

# Conteúdo do gabarito
conteudo_gabarito = ""
try:
    with open(caminho_gabarito, 'r', encoding='utf-8') as f:
        conteudo_gabarito = f.read()
except FileNotFoundError:
    print(f"Erro: Arquivo de gabarito não encontrado em '{caminho_gabarito}'")
    exit() # Sai do script se o gabarito não for encontrado
except Exception as e:
    print(f"Erro ao ler o arquivo de gabarito '{caminho_gabarito}': {e}")
    exit()


# Lista para armazenar o conteúdo dos outros textos
textos_para_comparar = []
# Lista para armazenar os nomes dos arquivos (para referência na saída)
nomes_arquivos_para_comparar = []

# Percorre todos os arquivos no diretório, exceto o gabarito
for nome_arquivo in os.listdir(diretorio_textos):
    caminho_completo = os.path.join(diretorio_textos, nome_arquivo)
    # Verifica se o arquivo termina com .txt e NÃO é o arquivo do gabarito
    if nome_arquivo.endswith('.txt') and caminho_completo != caminho_gabarito:
        try:
            # Abre e lê o conteúdo do arquivo
            with open(caminho_completo, 'r', encoding='utf-8') as f:
                conteudo = f.read()
                textos_para_comparar.append(conteudo)
                nomes_arquivos_para_comparar.append(nome_arquivo)
        except Exception as e:
            print(f"Erro ao ler o arquivo {nome_arquivo}: {e}")


# Verifica se há textos para comparar
if not textos_para_comparar:
    print("Nenhum arquivo .txt encontrado para comparação no diretório especificado (excluindo o gabarito).")
else:
    # Lista de stop words em português (a mesma lista do exemplo anterior)
    stop_words_portugues = [
        'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
        'de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'não', 'uma', 'os', 'as', 'dos', 'das', 'pelo', 'pela', 'pelos', 'pelas',
        'até', 'mais', 'como', 'ao', 'aos', 'à', 'às', 'outro', 'outra', 'outros', 'outras', 'este', 'esta', 'estes', 'estas', 'esse', 'essa', 'esses',
        'essas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'isso', 'aquilo', 'ele', 'ela', 'eles', 'elas', 'se', 'ou', 'quando', 'muito', 'qual',
        'quais', 'nos', 'nas', 'no', 'na', 'dele', 'dela', 'deles', 'delas', 'meu', 'minha', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'seu', 'sua',
        'seus', 'suas', 'nosso', 'nossa', 'nossos', 'nossas', 'vosso', 'vossa', 'vossos', 'vossas', 'por', 'desde', 'sem', 'entre', 'durante', 'perante',
        'após', 'sob', 'sobre', 'trás', 'àquele', 'àquela', 'àqueles', 'àquelas', 'desse', 'dessa', 'desses', 'dessas', 'deste', 'desta', 'destes',
        'destas', 'àquele', 'àquela', 'àqueles', 'àquelas', 'isso', 'isto', 'aquilo', 'tu', 'vós', 'lhes', 'me', 'te', 'lhe', 'nos', 'vos', 'lhes',
        'quem', 'onde', 'porque', 'que', 'talvez', 'assim', 'pois', 'logo', 'portanto', 'entretanto', 'contudo', 'todavia', 'ora', 'já', 'agora',
        'ainda', 'depois', 'antes', 'durante', 'enquanto', 'quando', 'se', 'senão', 'embora', 'conforme', 'segundo', 'desde', 'para', 'por', 'perante',
        'após', 'sob', 'sobre', 'trás', 'com', 'sem', 'de', 'em', 'entre', 'até', 'desde', 'ante', 'após', 'com', 'contra', 'de', 'desde', 'em',
        'entre', 'para', 'perante', 'por', 'sem', 'sob', 'sobre', 'trás', 'a', 'o', 'as', 'os', 'um', 'uma', 'uns', 'umas'
    ]

    # Inicializa o vetorizador TF-IDF com stop words em português
    vectorizer = TfidfVectorizer(stop_words=stop_words_portugues)

    # Cria uma lista contendo o gabarito e todos os outros textos
    todos_os_textos = [conteudo_gabarito] + textos_para_comparar

    # Transforma todos os textos em vetores TF-IDF
    vetores_tfidf = vectorizer.fit_transform(todos_os_textos)

    # O primeiro vetor (índice 0) corresponde ao gabarito
    vetor_gabarito = vetores_tfidf[0]

    # Os vetores restantes correspondem aos textos para comparar
    vetores_para_comparar = vetores_tfidf[1:]

    # Calcula a similaridade de cosseno entre o vetor do gabarito e os outros vetores
    similaridades = cosine_similarity(vetor_gabarito, vetores_para_comparar)

    # Exibe os resultados
    print(f"Similaridade do Gabarito ('{os.path.basename(caminho_gabarito)}') com outros arquivos:")
    # A matriz de similaridade terá uma linha e o número de colunas igual ao número de textos comparados
    for i in range(len(nomes_arquivos_para_comparar)):
        # similaridades[0][i] acessa a similaridade entre o gabarito (primeira linha) e o i-ésimo texto para comparar (coluna i)
        print(f"- Arquivo '{nomes_arquivos_para_comparar[i]}': {similaridades[0][i]:.2f}")

Similaridade do Gabarito ('gabarito.txt') com outros arquivos:
- Arquivo 'anthropic-claude-3.7-sonnet_7611-067.txt': 0.78
- Arquivo 'openai-gpt-4o-mini_7611-067.txt': 0.27
- Arquivo 'google-gemini-2.0-flash-001_7611-067.txt': 0.99
- Arquivo 'google-gemini-2.5-flash-preview_7611-067.txt': 0.76
- Arquivo 'google-gemini-2.5-pro-preview_7611-067.txt': 0.93
- Arquivo 'deepseek-deepseek-chat-v3-0324:free_7611-067.txt': 0.00
- Arquivo 'deepseek-deepseek-chat-v3-0324_7611-067.txt': 0.02
- Arquivo 'google-gemini-2.5-flash-preview:thinking_7611-067.txt': 0.78
- Arquivo 'anthropic-claude-3.7-sonnet:thinking_7611-067.txt': 0.77
- Arquivo 'openai-gpt-4.1_7611-067.txt': 0.44
